# Sample preprocessing

In this notebook we are going to preprocess the CellRange raw h5 files to create a preprocessed h5 adata file. The preprocessing consists on the following steps:
* QC
  * Cell selection
  * Mitochondrial content filtering
  * Count-based filtering
* Scrublet doublet detection
* SCRAN Normalization

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
from anndata import AnnData
import scanpy.external as sce
import matplotlib as mpl
import seaborn as sns
import scipy.sparse as spr

import rpy2.rinterface_lib.callbacks
import logging

from rpy2.robjects import pandas2ri
import anndata2ri

import scrublet as scr

In [ ]:
from scripts.preprocessing_scripts import cell_selection, cell_selection_plotting, plot_mito, run_scrublet, prepare_norm, apply_norm, adata_preprocessing

In [ ]:
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
sc.set_figure_params(dpi=200, dpi_save=300)

In [ ]:
%%R
# Load all the R libraries we will be using in the notebook
library(scran)
library(sctransform)

In [ ]:
seed = 10

# Selection of palettes for cluster coloring, and scatter values
magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 20)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:17])

# Discrete palette [Combination of BOLD and VIVID from carto colors]
bold_and_vivid = ['#7F3C8D','#11A579','#3969AC','#F2B701','#E73F74','#80BA5A','#E68310','#008695','#CF1C90',
           '#f97b72','#4b4b8f', '#E58606','#5D69B1','#52BCA3','#99C945','#CC61B0','#24796C','#DAA51B',
           '#2F8AC4','#764E9F','#ED645A','#CC3A8E']

prism = ['#5F4690', '#1D6996', '#38A6A5', '#0F8554', '#73AF48', '#EDAD08', '#E17C05', '#CC503E', '#94346E', '#6F4070', '#994E95']
prism = prism[::2] + prism[1::2]
safe = ['#88CCEE', '#CC6677', '#DDCC77', '#117733', '#332288', '#AA4499', '#44AA99', '#999933', '#882255', '#661100', '#6699CC']
vivid = ['#E58606', '#5D69B1', '#52BCA3', '#99C945', '#CC61B0', '#24796C', '#DAA51B', '#2F8AC4', '#764E9F', '#ED645A', '#CC3A8E']
bold = ['#7F3C8D', '#11A579', '#3969AC', '#F2B701', '#E73F74', '#80BA5A', '#E68310', '#008695', '#CF1C90', '#f97b72', '#4b4b8f']
# Diverging palettes
temps = ['#009392', '#39b185', '#9ccb86', '#e9e29c', '#eeb479', '#e88471', '#cf597e']

# Continuous palettes
teal = ['#d1eeea', '#a8dbd9', '#85c4c9', '#68abb8', '#4f90a6', '#3b738f', '#2a5674']

# Load CellRanger adatas

In [ ]:
dir_CR = os.getcwd() + '/data/CR'

In [ ]:
dir_preprocessed = os.getcwd() + '/data/preprocessed'
os.makedirs(dir_preprocessed, exist_ok=True)

In [ ]:
adata_Ap11 = sc.read_10x_h5(f'{dir_CR}/SI-GA-D12_Ap11/outs/raw_feature_bc_matrix.h5')
adata_Ap11.var_names_make_unique()

adata_Mp11 = sc.read_10x_h5(f'{dir_CR}/SI-GA-C10_Mp11/outs/raw_feature_bc_matrix.h5')
adata_Mp11.var_names_make_unique()

adata_Ap13 = sc.read_10x_h5(f'{dir_CR}/SI-GA-G7_Ap13/outs/raw_feature_bc_matrix.h5')
adata_Ap13.var_names_make_unique()

adata_Mp13 = sc.read_10x_h5(f'{dir_CR}/SI-GA-E10_Mp13/outs/raw_feature_bc_matrix.h5')
adata_Mp13.var_names_make_unique()

adata_Ap15 = sc.read_10x_h5(f'{dir_CR}/SI-GA-D8_Ap15/outs/raw_feature_bc_matrix.h5')
adata_Ap15.var_names_make_unique()

adata_Mp15 = sc.read_10x_h5(f'{dir_CR}/SI-GA-D10_Mp15/outs/raw_feature_bc_matrix.h5')
adata_Mp15.var_names_make_unique()

# Ap11

In [ ]:
# QC - cell selection
cell_selection_plotting(adata_Ap11, [4500, 4000, 3500, 3000], palette=bold, cmap=magma)
adata_Ap11 = cell_selection(adata_Ap11, selected_n_cells=3100, MALAT1_threshold=2.2)

In [ ]:
# QC - mitochondrial content filtering
adata_Ap11 = plot_mito(adata_Ap11)
adata_Ap11 = adata_Ap11[(adata_Ap11.obs.n_genes_by_counts < 3000) & (adata_Ap11.obs.percent_mito < 0.13), :]

In [ ]:
# scrublet
adata_Ap11 = run_scrublet(adata_Ap11, min_dist=0.1)

In [ ]:
# Normalization
data_mat, input_groups = prepare_norm(adata_Ap11, n_comps=50, resolution=0.1, seed=seed)

In [ ]:
%%R -i data_mat -i input_groups -o size_factors

size_factors = BiocGenerics::sizeFactors(scran::computeSumFactors(SingleCellExperiment::SingleCellExperiment(list(counts=data_mat)), clusters=input_groups))

In [ ]:
adata_Ap11 = apply_norm(adata_Ap11, size_factors)

In [ ]:
# Basic adata preprocessing to save for later
adata_preprocessing(adata_Ap11, sample='Ap11', hvg_min_mean=0.01, hvg_max_mean=5, hvg_min_disp=0.2, seed=seed, leiden_resolution=0.1, 
                    dir_adata_save=dir_preprocessed, cmap=magma, palette=bold_and_vivid)

# Mp11

In [ ]:
# QC - cell selection
cell_selection_plotting(adata_Mp11, [4500, 4000, 3500, 3000], palette=bold, cmap=magma)
adata_Mp11 = cell_selection(adata_Mp11, selected_n_cells=3200, MALAT1_threshold=3)

In [ ]:
# QC - mitochondrial content filtering
adata_Mp11 = plot_mito(adata_Mp11)
adata_Mp11 = adata_Mp11[(adata_Mp11.obs.n_genes_by_counts < 2700) & (adata_Mp11.obs.percent_mito < 0.1), :]

In [ ]:
# scrublet
adata_Mp11 = run_scrublet(adata_Mp11, min_dist=0.01)

In [ ]:
# Normalization
data_mat, input_groups = prepare_norm(adata_Mp11, n_comps=50, resolution=0.1, seed=seed)

In [ ]:
%%R -i data_mat -i input_groups -o size_factors

size_factors = BiocGenerics::sizeFactors(scran::computeSumFactors(SingleCellExperiment::SingleCellExperiment(list(counts=data_mat)), clusters=input_groups))

In [ ]:
adata_Mp11 = apply_norm(adata_Mp11, size_factors)

In [ ]:
# Basic adata preprocessing to save for later
adata_preprocessing(adata_Mp11, sample='Mp11', hvg_min_mean=0.01, hvg_max_mean=5, hvg_min_disp=0.2, seed=seed, leiden_resolution=0.1, 
                    dir_adata_save=dir_preprocessed, cmap=magma, palette=bold_and_vivid)

# Ap13

In [ ]:
# QC - cell selection
cell_selection_plotting(adata_Ap13, [4000, 3500, 3000, 2500], palette=bold, cmap=magma)
adata_Ap13 = cell_selection(adata_Ap13, selected_n_cells=3100, MALAT1_threshold=3.3)

In [ ]:
# QC - mitochondrial content filtering
adata_Ap13 = plot_mito(adata_Ap13)
adata_Ap13 = adata_Ap13[(adata_Ap13.obs.n_genes_by_counts < 2700) & (adata_Ap13.obs.percent_mito < 0.1), :]

In [ ]:
# scrublet
adata_Ap13 = run_scrublet(adata_Ap13, min_dist=0.01)

In [ ]:
# Normalization
data_mat, input_groups = prepare_norm(adata_Ap13, n_comps=50, resolution=0.3, seed=seed)

In [ ]:
%%R -i data_mat -i input_groups -o size_factors

size_factors = BiocGenerics::sizeFactors(scran::computeSumFactors(SingleCellExperiment::SingleCellExperiment(list(counts=data_mat)), clusters=input_groups))

In [ ]:
adata_Ap13 = apply_norm(adata_Ap13, size_factors)

In [ ]:
# Basic adata preprocessing to save for later
adata_preprocessing(adata_Ap13, sample='Ap13', hvg_min_mean=0.01, hvg_max_mean=5, hvg_min_disp=0, seed=seed, leiden_resolution=0.3, 
                    dir_adata_save=dir_preprocessed, cmap=magma, palette=bold_and_vivid)

# Mp13

In [ ]:
# QC - cell selection
cell_selection_plotting(adata_Mp13, [4500, 4000, 3500, 3000], palette=bold, cmap=magma)
adata_Mp13 = cell_selection(adata_Mp13, selected_n_cells=1200, MALAT1_threshold=3.2)

In [ ]:
# QC - mitochondrial content filtering
adata_Mp13 = plot_mito(adata_Mp13)
adata_Mp13 = adata_Mp13[(adata_Mp13.obs.n_genes_by_counts < 3000) & (adata_Mp13.obs.percent_mito < 0.13), :]

In [ ]:
# scrublet
adata_Mp13 = run_scrublet(adata_Mp13, min_dist=0.01)

In [ ]:
# Normalization
data_mat, input_groups = prepare_norm(adata_Mp13, n_comps=50, resolution=0.3, seed=seed)

In [ ]:
%%R -i data_mat -i input_groups -o size_factors

size_factors = BiocGenerics::sizeFactors(scran::computeSumFactors(SingleCellExperiment::SingleCellExperiment(list(counts=data_mat)), clusters=input_groups))

In [ ]:
adata_Mp13 = apply_norm(adata_Mp13, size_factors)

In [ ]:
# Basic adata preprocessing to save for later
adata_preprocessing(adata_Mp13, sample='Mp13', hvg_min_mean=0.01, hvg_max_mean=5, hvg_min_disp=0.2, seed=seed, leiden_resolution=0.3, 
                    dir_adata_save=dir_preprocessed, cmap=magma, palette=bold_and_vivid)

# Ap15

In [ ]:
# QC - cell selection
cell_selection_plotting(adata_Ap15, [3000, 2500, 2000, 1500], palette=bold, cmap=magma)
adata_Ap15 = cell_selection(adata_Ap15, selected_n_cells=1500, MALAT1_threshold=3)

In [ ]:
# QC - mitochondrial content filtering
adata_Ap15 = plot_mito(adata_Ap15)
adata_Ap15 = adata_Ap15[(adata_Ap15.obs.n_genes_by_counts < 1700) & (adata_Ap15.obs.percent_mito < 0.15), :]

In [ ]:
# scrublet
adata_Ap15 = run_scrublet(adata_Ap15, min_dist=0.01)

In [ ]:
# Normalization
data_mat, input_groups = prepare_norm(adata_Ap15, n_comps=50, resolution=0.3, seed=seed)

In [ ]:
%%R -i data_mat -i input_groups -o size_factors

size_factors = BiocGenerics::sizeFactors(scran::computeSumFactors(SingleCellExperiment::SingleCellExperiment(list(counts=data_mat)), clusters=input_groups))

In [ ]:
adata_Ap15 = apply_norm(adata_Ap15, size_factors)

In [ ]:
# Basic adata preprocessing to save for later
adata_preprocessing(adata_Ap15, sample='Ap15', hvg_min_mean=0.01, hvg_max_mean=5, hvg_min_disp=0, seed=seed, leiden_resolution=0.3, 
                    dir_adata_save=dir_preprocessed, cmap=magma, palette=bold_and_vivid)

# Mp15

In [ ]:
# QC - cell selection
cell_selection_plotting(adata_Mp15, [4500, 4000, 3500, 3000], palette=bold, cmap=magma)
adata_Mp15 = cell_selection(adata_Mp15, selected_n_cells=3500, MALAT1_threshold=3.5)

In [ ]:
# QC - mitochondrial content filtering
adata_Mp15 = plot_mito(adata_Mp15)
adata_Mp15 = adata_Mp15[(adata_Mp15.obs.n_genes_by_counts < 1700) & (adata_Mp15.obs.percent_mito < 0.13), :]

In [ ]:
# scrublet
adata_Mp15 = run_scrublet(adata_Mp15, min_dist=0.01)

In [ ]:
# Normalization
data_mat, input_groups = prepare_norm(adata_Mp15, n_comps=50, resolution=0.3, seed=seed)

In [ ]:
%%R -i data_mat -i input_groups -o size_factors

size_factors = BiocGenerics::sizeFactors(scran::computeSumFactors(SingleCellExperiment::SingleCellExperiment(list(counts=data_mat)), clusters=input_groups))

In [ ]:
adata_Mp15 = apply_norm(adata_Mp15, size_factors)

In [ ]:
# Basic adata preprocessing to save for later
adata_preprocessing(adata_Mp15, sample='Mp15', hvg_min_mean=0.01, hvg_max_mean=5, hvg_min_disp=0.2, seed=seed, leiden_resolution=0.1, 
                    dir_adata_save=dir_preprocessed, cmap=magma, palette=bold_and_vivid)